## Демонстрационный эксперимент в Google Colab (до 5 минут)

### Цель
Показать воспроизводимый протокол экспериментального сравнения фреймворков тестирования ML/данных на одной задаче прогнозирования временного ряда.

### Объект сравнения
Фреймворки:
- Great Expectations
- Evidently
- Alibi Detect
- NannyML

### Входные данные
Файлы находятся в `data/` (CSV/JSON). Используются колонки: `ds` (datetime), `y` (target), регрессоры `price`, `promotion`.

### Сценарии (синтетика)
Сценарии и “внедрённые” дефекты задаются в `generate_scenarios.py` и сохраняются в `data/`:
- `ideal`: контрольный сценарий без внедрённых проблем.
- `pass`: пропуски + дубликаты.
- `dr`: дрейф + выбросы (вводятся преимущественно в тестовом хвосте).

### Протокол (фиксированные правила)
- **Time split 80/20**: данные сортируются по `ds`, тест = последние 20% строк (см. `main.py`).
- Запуск пайплайна: обучение Prophet → прогноз `yhat` → запуск адаптеров фреймворков → сохранение артефактов.

### Выходные артефакты
Каждый запуск создаёт `reports/<run_name>/<timestamp>/...`:
- `comparison_summary.json` (унифицированные результаты по фреймворкам)
- `dashboard.html` (сравнительный HTML-дашборд)
- `final_summary.json`, `model_metrics.json` и артефакты фреймворков в подпапках

### Mini-table (E_test vs EF_test → итоговая таблица)
Дополнительно строится компактная сравнительная таблица на **test split**:
- `reports/mini_table_E_test.json` — количество и множества “внедрённых” дефектов (E_test)
- `reports/mini_table_EF_test.csv` / `.json` — детекции из артефактов фреймворков (EF_test)
- `reports/mini_table_final.csv` — EF_total / EF_true / FP / Recall / Precision / FalseAlarm
- `reports/mini_table_provenance.md` — трассировка источников EF_* (какие файлы артефактов используются)

Обозначения:
- **NA** — метрика не формировалась данным средством в текущей конфигурации.
- **—** — метрика неприменима (например, Recall при E_test=0; Precision при EF_total=0).

Репозиторий: `https://github.com/alexandor09/ml-testing-frameworks-comparison/`



In [ ]:
import os
import sys
import subprocess
from pathlib import Path

# Делаем ячейку идемпотентной: всегда стартуем из /content,
# чтобы повторный запуск не “углублял” cwd.
BASE_DIR = Path("/content")
BASE_DIR.mkdir(parents=True, exist_ok=True)
os.chdir(BASE_DIR)

REPO_URL = "https://github.com/alexandor09/ml-testing-frameworks-comparison.git"
REPO_DIR = "ml-testing-frameworks-comparison"

if not Path(REPO_DIR).exists():
    subprocess.check_call(["git", "clone", REPO_URL])

# Находим директорию, где лежит main.py (репо может содержать вложенную папку).
CANDIDATES = [
    BASE_DIR / REPO_DIR,
    BASE_DIR / REPO_DIR / REPO_DIR,
    BASE_DIR / REPO_DIR / REPO_DIR / REPO_DIR,
]
code_root = None
for c in CANDIDATES:
    if (c / "main.py").exists():
        code_root = c
        break

if code_root is None:
    raise FileNotFoundError(f"Cannot find main.py in candidates: {CANDIDATES}")

os.chdir(code_root)
print("cwd:", os.getcwd())
print("python:", sys.version)



In [ ]:
# Установка зависимостей (воспроизводимый эксперимент)
#
# В Colab иногда возникает ошибка вида:
#   ValueError: numpy.dtype size changed, may indicate binary incompatibility
# Это следствие рассинхронизации бинарных колёс numpy/pandas (после частичных обновлений).
# Надёжный порядок действий:
# 1) выполнить эту ячейку
# 2) Runtime → Restart runtime
# 3) Run all (с начала)
import sys
import subprocess

INSTALL_MODE = "full"  # "full" | "light"

def pip_install(*args: str) -> None:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", *args])

# Важно: no-cache-dir + force-reinstall, чтобы не подтягивались старые колёса.
pip_install("--upgrade", "pip")
pip_install("--no-cache-dir", "--upgrade", "--force-reinstall", "numpy", "pandas")

if INSTALL_MODE == "full":
    pip_install("--no-cache-dir", "--upgrade", "-r", "requirements.txt")
else:
    pip_install(
        "--no-cache-dir",
        "--upgrade",
        "pandas", "numpy", "prophet", "great_expectations", "psutil", "plotly", "scikit-learn", "tqdm",
    )

# Диагностика: показываем версии, не импортируя pandas (импорт может падать до Restart runtime)
subprocess.check_call([sys.executable, "-m", "pip", "show", "numpy", "pandas"])
print("\nIMPORTANT: Runtime → Restart runtime, then run the notebook again from the top.")



In [ ]:
from pathlib import Path

print("### data/")
data_dir = Path("data")
for p in sorted(data_dir.glob("*.csv")):
    print("-", p.as_posix())



### Определения: E_test и EF_test

- **E_test** — “истинные” (внедрённые) дефекты на тестовой части (последние 20% по времени).
  - Вычисляется скриптом `scripts/count_E_test.py` из самих данных (на test split).
- **EF_test** — дефекты, извлечённые из артефактов фреймворков на тестовой части.
  - Вычисляется скриптом `scripts/count_EF_test.py` по `comparison_summary.json` и артефактам в `reports/run_*/<timestamp>/<framework>/...`.

Сравнение выполняется в единых единицах (строки/фичи) и затем сводится в `scripts/build_mini_table.py`.



In [ ]:
import sys
import subprocess

# (Опционально) быстрый sanity-check: запуск одного фреймворка.
# По умолчанию отключено, чтобы Runtime→Run all выполнял основной эксперимент (ячейка mini-table).
RUN_QUICK_SINGLE_FRAMEWORK = False

INPUT = "data/dr.csv"
OUTPUT = "reports/colab_demo_dr"
FRAMEWORK = "gx"  # gx | evidently | alibi | nannyml

if RUN_QUICK_SINGLE_FRAMEWORK:
    subprocess.check_call([
        sys.executable, "main.py",
        "--input", INPUT,
        "--format", "csv",
        "--output", OUTPUT,
        "--framework", FRAMEWORK,
    ])
else:
    print("Quick single-framework run is disabled (RUN_QUICK_SINGLE_FRAMEWORK=False).")



In [ ]:
import json
from pathlib import Path

# Просмотр результатов «быстрого» прогона (если он включён в предыдущей ячейке).
run_base = Path("reports/colab_demo_dr")

if run_base.exists():
    runs = sorted([p for p in run_base.glob("*") if p.is_dir()])
else:
    runs = []

if not runs:
    print("No quick-run outputs found (run_base does not exist or empty).")
else:
    latest = runs[-1]
    print("Latest run:", latest.as_posix())

    print("\n### files")
    for p in sorted(latest.glob("*") ):
        print("-", p.name)

    summary_path = latest / "comparison_summary.json"
    print("\n### comparison_summary.json")
    print(summary_path.as_posix())
    print(json.dumps(json.loads(summary_path.read_text(encoding="utf-8")), indent=2, ensure_ascii=False)[:4000])



In [ ]:
import sys
import subprocess
from pathlib import Path
import json

# Важно для Colab: если после pip-установок рантайм не был перезапущен,
# возможна бинарная несовместимость numpy/pandas.
try:
    import pandas as pd
except Exception as e:
    if "numpy.dtype size changed" in str(e):
        raise RuntimeError(
            "Binary incompatibility between numpy and pandas. "
            "Fix: Runtime → Restart runtime, then run the notebook again from the top."
        ) from e
    raise

print("pandas:", pd.__version__)

# Полный эксперимент: прогон 3 сценариев (ideal/pass/dr) всеми 4 фреймворками
# и последующее построение mini-table.
SCENARIOS = [
    ("ideal", "data/ideal.csv", "reports/run_ideal"),
    ("pass",  "data/pass.csv",  "reports/run_pass"),
    ("dr",    "data/dr.csv",    "reports/run_dr"),
]

for name, inp, out in SCENARIOS:
    print(f"\n=== RUN: {name} ===")
    subprocess.check_call([
        sys.executable, "main.py",
        "--input", inp,
        "--format", "csv",
        "--output", out,
    ])

# Извлекаем сводные таблицы сравнения из comparison_summary.json

def _latest_run_dir(parent: str) -> Path:
    p = Path(parent)
    runs = sorted([x for x in p.glob("*") if x.is_dir()])
    if not runs:
        raise FileNotFoundError(f"No runs found in {parent}")
    return runs[-1]


def _load_summary(run_dir: Path) -> dict:
    return json.loads((run_dir / "comparison_summary.json").read_text(encoding="utf-8"))


def _summary_df(summary: dict, scenario: str) -> pd.DataFrame:
    rows = []
    for fw, res in (summary or {}).items():
        rows.append(
            {
                "scenario": scenario,
                "framework": fw,
                "time_sec": float(res.get("execution_time_sec", 0.0) or 0.0),
                "peak_rss_mb": float(res.get("memory_peak_mb", 0.0) or 0.0),
                "issues_detected": int(res.get("issues_detected", 0) or 0),
                "coverage": float(res.get("coverage_score", 0.0) or 0.0),
                "artifacts_n": int(len(res.get("artifacts", []) or [])),
                "data_quality": bool((res.get("checks_performed", {}) or {}).get("data_quality", False)),
                "data_drift": bool((res.get("checks_performed", {}) or {}).get("data_drift", False)),
                "outliers": bool((res.get("checks_performed", {}) or {}).get("outliers", False)),
                "model_perf": bool((res.get("checks_performed", {}) or {}).get("model_performance", False)),
            }
        )
    df = pd.DataFrame(rows)
    if len(df) == 0:
        return df
    return df.sort_values(["scenario", "framework"]).reset_index(drop=True)


print("\n=== COMPARISON TABLES (from comparison_summary.json) ===")
all_tables = []
for scen, _inp, out_parent in SCENARIOS:
    run_dir = _latest_run_dir(out_parent)
    summary = _load_summary(run_dir)
    df = _summary_df(summary, scenario=scen)
    all_tables.append(df)
    print(f"\nScenario: {scen}")
    display(df[["framework", "time_sec", "peak_rss_mb", "issues_detected", "coverage", "artifacts_n"]])

# Объединённая таблица (все сценарии вместе)
df_all = pd.concat(all_tables, ignore_index=True)
print("\nAll scenarios (stacked):")
display(df_all)

# Построение mini-table (E_test, EF_test, final)
print("\n=== MINI-TABLE PIPELINE ===")
subprocess.check_call([sys.executable, "scripts/count_E_test.py", "--inputs", "data/pass.csv", "data/dr.csv", "data/ideal.csv"])
subprocess.check_call([sys.executable, "scripts/count_EF_test.py"])
subprocess.check_call([sys.executable, "scripts/build_mini_table.py"])

print("\nSaved:")
print("- reports/mini_table_E_test.json")
print("- reports/mini_table_EF_test.json")
print("- reports/mini_table_EF_test.csv")
print("- reports/mini_table_final.csv")
print("- reports/mini_table_provenance.md")

print("\nPreview: reports/mini_table_final.csv")
df_final = pd.read_csv("reports/mini_table_final.csv")
display(df_final)

# (Опционально) скачивание dashboard.html для сценария dr
try:
    from google.colab import files  # type: ignore

    dr_run_dir = _latest_run_dir("reports/run_dr")
    dash_path = dr_run_dir / "dashboard.html"
    if dash_path.exists():
        print(f"\nDownloading dashboard: {dash_path.as_posix()}")
        files.download(dash_path.as_posix())
except Exception:
    # Вне Colab этот блок можно игнорировать
    pass



### Дополнительный запуск (один сценарий)

Основной эксперимент (прогоны `ideal/pass/dr` + сравнение + mini-table) выполняется в ячейке выше.

Ниже оставлена дополнительная ячейка для запуска **одного** сценария (например, только `dr`) и получения `dashboard.html`.



In [ ]:
# Дополнительный запуск одного сценария (отдельная папка)
# По умолчанию отключено, чтобы не тратить время при Runtime→Run all.

import sys
import subprocess
import json
from pathlib import Path

RUN_EXTRA_SINGLE_SCENARIO = False

INPUT_FULL = "data/dr.csv"
OUTPUT_FULL = "reports/colab_demo_full"

if RUN_EXTRA_SINGLE_SCENARIO:
    subprocess.check_call([
        sys.executable, "main.py",
        "--input", INPUT_FULL,
        "--format", "csv",
        "--output", OUTPUT_FULL,
    ])

    run_base = Path(OUTPUT_FULL)
    runs = sorted([p for p in run_base.glob("*") if p.is_dir()])
    latest = runs[-1]
    print("Latest full run:", latest.as_posix())

    print("\nDashboard:", (latest / "dashboard.html").as_posix())
    print("Final summary:", (latest / "final_summary.json").as_posix())

    # В Colab можно скачать HTML и открыть локально в браузере.
    try:
        from google.colab import files  # type: ignore

        files.download((latest / "dashboard.html").as_posix())
    except Exception:
        pass
else:
    print("Extra single-scenario run is disabled (RUN_EXTRA_SINGLE_SCENARIO=False).")



### Mini-table (E_test vs EF_total/EF_true/FP + FalseAlarm)

Требует `INSTALL_MODE = "full"` (потому что нужны артефакты всех 4 фреймворков).

Пайплайн:
- запускаем `ideal`, `pass`, `dr` → артефакты в `reports/run_*/<timestamp>/...`
- считаем `E_test`, `EF_test`, собираем финальную таблицу

Важно:
- В итоговой таблице есть `n_test` (контекст размера теста).
- **NA** = метрика не формировалась данным средством в текущей конфигурации.
- **—** = метрика неприменима (например, Recall при `E_test=0`; Precision при `EF_total=0` или при `E_test=0`).
- `FalseAlarm` выводится только когда `E_test=0`:
  - для строковых дефектов: `EF_total / n_test`
  - для `drift_features`: `EF_total / 3`

